In [1]:
require 'daru'
require 'nyaplot'

N = 10
M = [0, 1, 3, 9]

def normal_rand(mu = 0, sigma = 1.0)
  random = Random.new
  (Math.sqrt(-2 * Math.log(random.rand)) * Math.sin(2 * Math::PI * random.rand) * sigma) + mu
end


def create_dataset(num)
  dataset = Daru::DataFrame.new({'x': [], 'y': []})  

  num.times do |i|
    x = i.to_f / (num - 1).to_f
    y = Math.sin(2 * Math::PI * x) + normal_rand(0, 0.3)
    dataset.add_row(Daru::Vector.new([x, y], index: [:x, :y]))
  end
  
  return dataset
end

def log_likelihood(dataset, f)
  dev = 0.0
  n = dataset.size.to_f
  dataset.each_row do |line|
    x = line.x
    y = line.y
    dev += (y - f.call(x))**2
  end
  err = dev * 0.5
  beta = n / dev
  -beta * err + 0.5 * n * Math.log(0.5 * beta / Math::PI)
end

def resolve(dataset, m)
  t = dataset.y

  columns = {}
  (m+1).times do |i|
    columns["x**#{i}"] = dataset.x ** i
  end
  phi = Daru::DataFrame.new(columns)

  tmp = (phi.transpose.to_matrix * phi.to_matrix).inv
  ws = (tmp * phi.transpose.to_matrix) * Vector.elements(t.to_a)

  f = lambda {|x|
    y = 0
    ws.each_with_index do |w, i|
      y += w * (x ** i)
    end

    y
  }

  sigma2 = 0.0
  dataset.each_row do |line|
    sigma2 += (f.call(line.x) - line.y)**2
  end
  sigma2 /= dataset.size

  return f, ws, Math.sqrt(sigma2)
end

train_set = create_dataset(N)
test_set = create_dataset(N)
df_ws = {}

fig = Nyaplot::Frame.new

M.each_with_index do |m, c|
  f, ws, sigma = resolve(train_set, m)
  df_ws["M=#{m}"] = Daru::Vector.new(ws, name: "M=#{m}")
  
  plot = Nyaplot::Plot.new
  sc = plot.add_with_df(train_set.to_nyaplotdf, :scatter, :x, :y)
  sc.title("train_set")
  sc.color('blue')
  
  linex = (0..1).step(1.0 / (101 - 1)).to_a
  liney = linex.map do |x|
    Math.sin(2 * Math::PI * x)
  end
  line_answer = plot.add(:line, linex, liney)
  line_answer.title('answer')
  line_answer.color('green')
  
  liney = linex.map do |x|
    f.call(x)
  end
  line_middle = plot.add(:line, linex, liney)
  line_middle.title("Sigma=#{sprintf("%.2f", sigma)}")
  line_middle.color('red')
  line_upper = plot.add(:line, linex, liney.map {|y| y + sigma })
  line_upper.color('red')
  line_lower = plot.add(:line, linex, liney.map {|y| y - sigma } )
  line_lower.color('red')

  plot.configure do
    x_label("M=#{m}")
    y_label('')
    xrange([-0.05, 1.05])
    yrange([-1.5, 1.5])
    legend(true)
    height(300)
    width(490)
  end
  
  fig.add(plot)
end

fig.show

df = Daru::DataFrame.new({m: [], 'Test set': [], 'Training set': []})  
9.times do |m|
  f, ws, sigma = resolve(train_set, m)
  train_mlh = log_likelihood(train_set, f)
  test_mlh = log_likelihood(test_set, f)
  df.add_row(Daru::Vector.new([m, test_mlh, train_mlh], index: [:m, 'Test set'.to_sym, 'Training set'.to_sym]))
end

df.plot(type: [:line, :line], x: [:m, :m], y: ['Test set'.to_sym, 'Training set'.to_sym]) do |plot, diagrams|
  test_set_diagram = diagrams[0]
  train_set_diagram = diagrams[1]
  
  train_set_diagram.title('Training set')
  train_set_diagram.color('blue')
  test_set_diagram.title('Test set')
  test_set_diagram.color('green')
  
  plot.x_label("Log likelihood for N=#{N}")
  plot.y_label('')
  plot.legend(true)
end


"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

#<Nyaplot::Frame:0x007f15ad967b38 @properties={:panes=>[#<Nyaplot::Plot:0x007f15ad8f02b8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f15ad88b480 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :title=>"train_set", :color=>"blue"}, :data=>"c769df94-a80a-4c3a-9739-2dca80c99e28"}, @xrange=[0.0, 1.0], @yrange=[-1.0123344200594788, 1.1649813416498827]>, #<Nyaplot::Diagram:0x007f15ad85d710 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"answer", :color=>"green"}, :data=>"e64d5ede-4b44-48ff-a1ac-c1eecd6cd3aa"}, @xrange=[0.0, 1.0], @yrange=[-1.0, 1.0]>, #<Nyaplot::Diagram:0x007f15ad6d7620 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"Sigma=0.71", :color=>"red"}, :data=>"087fd5d3-97d9-4862-ad1c-51d4e3dbc1d2"}, @xrange=[0.0, 1.0], @yrange=[0.020134066569945815, 0.020134066569945815]>, #<Nyaplot::Diagram:0x007f15ad69e348 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"red"}, :data=>"0c29b527-9362-44d8-965d-357094d4802f"}, @xrange=[0.0, 1.0], @yrange=[0.7273122500240135, 0.7273122500240135]>, #<Nyaplot::Diagram:0x007f15ad673288 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"red"}, :data=>"7fd33e4b-0eac-4faf-95cc-4f6501adc4b6"}, @xrange=[0.0, 1.0], @yrange=[-0.6870441168841219, -0.6870441168841219]>], :options=>{:x_label=>"M=0", :y_label=>"", :xrange=>[-0.05, 1.05], :yrange=>[-1.5, 1.5], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007f15ad530cb8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f15ad583a58 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :title=>"train_set", :color=>"blue"}, :data=>"0543fad3-864a-49a1-847f-e53c94783440"}, @xrange=[0.0, 1.0], @yrange=[-1.0123344200594788, 1.1649813416498827]>, #<Nyaplot::Diagram:0x007f15ad574300 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"answer", :color=>"green"}, :data=>"6a1a37e8-e90c-4e43-a804-11638a496e40"}, @xrange=[0.0, 1.0], @yrange=[-1.0, 1.0]>, #<Nyaplot::Diagram:0x007f15ad392c08 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"Sigma=0.56", :color=>"red"}, :data=>"08350eed-471e-49a9-82f1-dbe7521ccb2b"}, @xrange=[0.0, 1.0], @yrange=[-0.6609154798654558, 0.7011836130053476]>, #<Nyaplot::Diagram:0x007f15ad383d48 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"red"}, :data=>"a05099af-a25c-48bb-b1e7-94201e770579"}, @xrange=[0.0, 1.0], @yrange=[-0.10312177004065326, 1.2589773228301502]>, #<Nyaplot::Diagram:0x007f15ae1574b0 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"red"}, :data=>"9f861927-bb55-402d-9785-ee90ff90b67d"}, @xrange=[0.0, 1.0], @yrange=[-1.2187091896902584, 0.143389903180545]>], :options=>{:x_label=>"M=1", :y_label=>"", :xrange=>[-0.05, 1.05], :yrange=>[-1.5, 1.5], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007f15ae0ba430 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f15ae0ae040 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :title=>"train_set", :color=>"blue"}, :data=>"3d1932b6-86e4-4c3f-9d1d-d7e05c815c4a"}, @xrange=[0.0, 1.0], @yrange=[-1.0123344200594788, 1.1649813416498827]>, #<Nyaplot::Diagram:0x007f15ae0a66b0 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"answer", :color=>"green"}, :data=>"22178b8b-aabf-4c67-9759-6d76dbea8df7"}, @xrange=[0.0, 1.0], @yrange=[-1.0, 1.0]>, #<Nyaplot::Diagram:0x007f15ae094bb8 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"Sigma=0.12", :color=>"red"}, :data=>"7e48b3be-e161-4ade-b2ec-ee63473c90f1"}, @xrange=[0.0, 1.0], @yrange=[-1.0205549917932917, 1.0324971584160134]>, #<Nyaplot::Diagram:0x007f15ae0898d0 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"red"}, :data=>"55ba9584-a319-47b6-9422-274eb960d2ac"}, @xrange=[0.0, 1.0], @yrange=[-0.8966567185955997, 1.1563954316137055]>, #<Nyaplot::Diagram:0x007f15ae07dee0 @properties={:type=>:line, :options=>{:x=>"data0", :

#<Nyaplot::Frame:0x007f15ad6d19f0 @properties={:panes=>[#<Nyaplot::Plot:0x007f15ad6b90a8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f15ad6fc5b0 @properties={:type=>:line, :options=>{:x=>:m, :y=>:"Test set", :title=>"Test set", :color=>"green"}, :data=>"109907ad-2330-4a2c-ab16-8723fdf8e7c8"}, @xrange=[0, 8], @yrange=[-10.959248334611097, 6.22863542433814]>, #<Nyaplot::Diagram:0x007f15ad6e7a48 @properties={:type=>:line, :options=>{:x=>:m, :y=>:"Training set", :title=>"Training set", :color=>"blue"}, :data=>"109907ad-2330-4a2c-ab16-8723fdf8e7c8"}, @xrange=[0, 8], @yrange=[-10.724659158818707, 54.78957627143174]>], :options=>{:x_label=>"Log likelihood for N=10", :y_label=>"", :legend=>true, :zoom=>true, :width=>800, :xrange=>[0, 8], :yrange=>[-10.959248334611097, 54.78957627143174]}}>], :data=>{"109907ad-2330-4a2c-ab16-8723fdf8e7c8"=>#<Nyaplot::DataFrame:0x007f15ad6fd5c8 @name="109907ad-2330-4a2c-ab16-8723fdf8e7c8", @rows=[{:"Test set"=>-10.959248334611097, :"Training set"=>-10.724659158818707, :m=>0}, {:"Test set"=>-8.394954941428635, :"Training set"=>-8.35172452595085, :m=>1}, {:"Test set"=>-8.405406955434055, :"Training set"=>-8.293834872758055, :m=>2}, {:"Test set"=>6.22863542433814, :"Training set"=>6.69355894304122, :m=>3}, {:"Test set"=>6.227504317715638, :"Training set"=>6.6936262828487525, :m=>4}, {:"Test set"=>3.442481754400797, :"Training set"=>9.935165809299908, :m=>5}, {:"Test set"=>2.6903231988997343, :"Training set"=>12.664068141026764, :m=>6}, {:"Test set"=>2.7092083106614098, :"Training set"=>12.894157291800312, :m=>7}, {:"Test set"=>3.327909256286116, :"Training set"=>54.78957627143174, :m=>8}]>}, :extension=>[]}>